# Reading Images

FILES

https://github.com/ScryptIQ-ai/lesson-data/blob/main/dh/axon_slice.jpg
https://github.com/ScryptIQ-ai/lesson-data/blob/main/dh/rat_brain_low_res.jpg

PACKAGES

numpy
matplotlib

## Introduction

**Learning Objectives**
- Understanding images as mathmatical arrays
- Working with 2-dimensional greyscale images
- Learning how to read and plot images with matplotlib
- Understanding 2-dimensional colour images and RGB channels

**Reading and Processing Images**
In biology, we often deal with images; for example, microscopy and different medical imaging modalities often output images and image data for analysis. In many cases, we wish to extract some quantitative information from these images. The focus of this lesson is to read and process images in Python. 

This will include:
- Working with 2-dimensional greyscale images
- Creating and applying binary image masks
- Working with 2-dimensional colour images, and interpreting colour channels
- Decreasing the memory for further processing by reducing resolution or patching
- Working with 3-dimensional images

## Image Example

The example in Figure 1 is an image from the cell [image library](http://cellimagelibrary.org/home) with the following description:

```
"Midsagittal section of rat cerebellum, captured using confocal imaging. Section shows inositol trisphosphate receptor (IP3R) labelled in green, DNA in blue, and synaptophysin in magenta. Honorable Mention, 2010 Olympus BioScapes Digital Imaging Competition®."
```

Given this information, we might want to determine the relative amounts of IP3R, DNA and synaptophysin in this image. This tutorial will guide you through some of the steps to get you started with processing images of all types, using Python. You can then refer back to this image analysis example for the assignment, and perform some analyses of your own.

![Figure 1: Example image, rat cerebellum](./images/rat_cerebellum.jpeg)

## Reading and Plotting a 2-Dimensional Image

Firstly, we must read an image in. To demonstrate this we can use the example of a histological slice through an axon bundle. We use Matplotlib's image module, from which we import the function `imread`, used to store the image in a variable that we will name `img`. The function `imread` can interpret many different image formats, including .jpg, .png and .tif images.

VIDEO

https://www.youtube.com/watch?v=Wn2GCHu-Glw

In [ ]:
from matplotlib.image import imread

In [ ]:
# Place the path to your image in the imread function
img = imread('./data/axon_slice.jpg')

We can then check what type of variable it is, using the type() function:

In [ ]:
print(type(img))

This tells us that the image is stored in a NumPy array; `ndarray` here refers to an N-dimensional array. We can check some other properties of this array, for example, what the image dimensions are, using the .shape attribute:

In [ ]:
print(img.shape)

This tells us that our image is composed of 2300 by 3040 data units, or _pixels_ as we are dealing with an image. The total number of pixels in the image, is equivalent to its resolution. In digital imaging, a pixel is simply a numerical value that represents either intensity, in a greyscale image, or colour intensity, in a colour image. Therefore, as data, a two-dimensional (2-D) greyscale image comprises a 2-D array of these intensity values, or elements; hence the name 'pixel' is an amalgamation of 'picture' and 'element'. The array in our example has two dimensions, and so we can expect the image to be 2-D as well. Let us now use matplotlib.pyplot's `imshow` function to plot the image to see what it looks like. We can set the colour map to `gray` in order to overwrite the default colour map.

In [ ]:
from matplotlib.pyplot import subplots, show

In [ ]:
fig, ax = subplots(figsize=(25, 15))

ax.imshow(img, cmap='gray');

show()

`imshow` has allowed us to plot the NumPy array of our image data as a picture. The figure is divided up into a number of pixels, and each of these is assigned an intensity value, which is held in the NumPy array. Let's have a closer look by selecting a smaller region of our image and plotting that.

In [ ]:
fig, ax = subplots(figsize=(10, 14))

ax.imshow(img[:50, :70], cmap='gray');

show()

By using `img[:50, :70]` we can select the first 50 values from the first dimension, and the first 70 values from the second dimension. Note that using a colon denotes 'all', where `:50` is inclusive of every number up to 50, for example. Thus, the image above shows a very small part of the upper left corner of our original image. As we are now zoomed in quite close to that corner, we can easily see the individual pixels here.

PRACTICE EXERCISE

Have a practice slicing the same image in the cell below:

In [ ]:
# Edit the slice in the code below:

fig, ax = subplots(figsize=(10, 14))

# Add you values to the slice here: 
ax.imshow(img[:, :], cmap='gray');

show()

## Pixel values

Next, lets zoom in even further:

In [ ]:
fig, ax = subplots(figsize=(8, 6))

ax.imshow(img[:20, :15], cmap='gray');

show()

We have now displayed an even smaller section from that same upper left corner of the image. Each square is a pixel and it has a single grey value. Thus, the pixel intensity values are assigned by the numbers stored in the NumPy array, `img`. Let us have a look at these values by producing a slice from the array.

In [ ]:
print(img[:20, :15])

Each of these numbers corresponds to an intensity in the specified colourmap. These numbers range from 0 to 255, implying 256 shades of grey.

We chose `cmap = gray`, which assigns darker grey colours to smaller numbers, and lighter grey colours to higher numbers. However, we can also pick a non-greyscale colour map to plot the same image. We can even display a colourbar to keep track of the intensity values. Matplotlib has a diverse palette of colour maps that you can look through, linked [here](https://matplotlib.org/tutorials/colors/colormaps.html). Let's take a look at two commonly-used Matplotlib colour maps called `viridis` and `magma`:

In [ ]:
fig, ax = subplots(nrows=1, ncols=2, figsize=(20, 10))

p1 = ax[0].imshow(img[:20, :15], cmap='viridis')
p2 = ax[1].imshow(img[:20, :15], cmap='magma')
fig.colorbar(p1, ax=ax[0], shrink = 0.8)
fig.colorbar(p2, ax=ax[1], shrink = 0.8);

show()

Note, that even though we can plot our greyscale image with colourful colour schemes, it still does not qualify as a colour image. These are just false-colour, Matplotlib-applied interpretations of rudimentary intensity values. The reason for this is that a pixel in a standard colour image actually comprises **three sets** of intensities per pixel; not just the one, as in this greyscale example. In the case above, the number in the array represented a grey value and the colour was assigned to that grey value by `Matplotlib`. These represent 'false' colours.

## Colour Images

VIDEO

https://www.youtube.com/watch?v=JVx2szMOZc8

So far, our image has had a single intensity value for each pixel. In colour images, we have multiple channels of information per pixel; most commonly, colour images are RGB, comprising three corresponding to intensities of red, green and blue. There are many other formats of colour images, but these are beyond the scope of this lesson. In a RGB colour image, any colour will be a composite of the intensity value for each of these colours. Our chosen example for a colour image is that of a rat cerebellar cortex. Let us firstly import it and check its shape.

In [ ]:
img_col = imread('./data/rat_brain_low_res.jpg')

print(img_col.shape)

Our image array now contains three dimensions: the first two are the spatial dimensions corresponding to the pixel positions as x-y coordinates. The third dimension contains the three colour channels, corresponding to three layers of intensity values (red, blue and green) on top of each other, per pixel.

First, let us plot the whole image.

In [ ]:
fig, ax = subplots(figsize=(25, 15))

ax.imshow(img_col);

show()

The sample is labeled for Hoechst stain (blue), the Inositol trisphosphate (IP3) receptor (green) and Glial fibrillary acidic protein (GFAP) (red).

We can then visualise the three colour channels individually by slicing the NumPy array comprising our image. The stack with index 0 corresponds to 'red', index 1 corresponds to 'green' and index 2 corresponds to 'blue':

In [ ]:
red_channel   = img_col[:, :, 0]
green_channel = img_col[:, :, 1]
blue_channel  = img_col[:, :, 2]

In [ ]:
fig, ax = subplots(nrows=1, ncols=3, figsize=(20, 10))

imgplot_red = ax[0].imshow(red_channel, cmap="Reds")
imgplot_green = ax[1].imshow(green_channel, cmap="Greens")
imgplot_blue = ax[2].imshow(blue_channel, cmap="Blues")

fig.colorbar(imgplot_red, ax=ax[0], shrink=0.4)
fig.colorbar(imgplot_green, ax=ax[1], shrink=0.4)
fig.colorbar(imgplot_blue, ax=ax[2], shrink=0.4);

show()

This shows what colour combinations each of the pixels is made up of. Notice that the intensities go up to 255. This is because RGB (red, green and blue) colours are defined within an intensity range of 0-255. This gives a vast total 16,777,216 possible colour combinations.

We can plot histograms of each of the colour channels.

In [ ]:
fig, ax = subplots(nrows=1, ncols=3, figsize=(20, 5))

ax[0].hist(red_channel.flatten(), bins=50)
ax[0].set_xlabel("Red channel")
ax[1].hist(green_channel.flatten(), bins=50)
ax[1].set_xlabel("Green channel")
ax[2].hist(blue_channel.flatten(), bins=50)
ax[2].set_xlabel("Blue channel")

show()

## Summary

Approaching images as numerical data rather than static pictures unlocks new and powerful ways to analyse them, quantitatively. By learning how images are stored as arrays of pixel intensities, how the latter map to brightness in a given channel, and how colour information is separated into three (sometimes four) channels, allows us to interrogate visual data with the same tools used for any other scientific dataset. This allows us to powerfully and iteratively extract meaningful biological information from images, enabling downstream tasks such as comparison, measurement and more advanced image analysis workflows.